In [8]:
GENIUS_API_TOKEN='F2SBCBsZrWPf0Isy6wKwnYyVDpZF9z67Zzd_UjHPNvEylkr0j6D0Qo0sQMeCGfW9'

In [21]:
import requests
from bs4 import BeautifulSoup
import os
import re
import pickle
import numpy as np

In [15]:
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response
# Get Genius.com song url's from artist object
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()
        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print('Found {} songs by {}'.format(len(songs), artist_name))
    return songs
    
# DEMO
request_song_url('Lana Del Rey', 2)

Found 2 songs by Lana Del Rey


['https://genius.com/Lana-del-rey-young-and-beautiful-lyrics',
 'https://genius.com/Lana-del-rey-summertime-sadness-lyrics']

In [11]:
# Scrape lyrics from a Genius.com song URL
def scrape_song_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    # lyrics = html.find('div', {'class': "lyrics"}).get_text()
    for tag in html.select('div[class^="Lyrics__Container"], .song_body-lyrics p'):
        lyrics = tag.get_text(strip=True, separator='\n')
        if lyrics:
            print(lyrics)
    #remove identifiers like chorus, verse, etc
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    #remove empty lines
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])         
    return lyrics
# DEMO
print(scrape_song_lyrics('https://genius.com/Lana-del-rey-young-and-beautiful-lyrics'))

[Verse 1]
I've seen the world, done it all
Had my cake now
Diamonds, brilliant,
and Bel Air now
Hot summer nights, mid-July
When you and I were forever wild
The crazy days, city lights
The way you'd play with me like a child
[Chorus]
Will you still love me when I'm no longer young and beautiful?
Will you still love me when I got nothing but my aching soul?
I know you will, I know you will, I know that you will
Will you still love me when I'm no longer beautiful?
[Verse 2]
I've seen the world, lit it up as my stage now
Channeling angels in the new age now
Hot summer days
, rock and roll
The way you'd play for me at your show
And all the ways I got to know
Your pretty face and electric soul
[Chorus]
Will you still love me when I'm no longer young and beautiful?
Will you still love me when I got nothing but my aching soul?
I know you will, I know you will, I know that you will
Will you still love me when I'm no longer beautiful?
[Bridge]
Dear Lord, when I get to heaven
Please, let me brin

In [16]:
def write_lyrics_to_file(artist_name, song_count):
    f = open('lyrics/' + artist_name.lower() + '.txt', 'wb')
    urls = request_song_url(artist_name, song_count)
    for url in urls:
        lyrics = scrape_song_lyrics(url)
        f.write(lyrics.encode("utf8"))
    f.close()
    num_lines = sum(1 for line in open('lyrics/' + artist_name.lower() + '.txt', 'rb'))
    print('Wrote {} lines to file from {} songs'.format(num_lines, song_count))

In [13]:
write_lyrics_to_file('Kendrick Lamar', 100)

Found 100 songs by Kendrick Lamar
[Produced by Hit-Boy]
[Intro]
Uh, Martin had a dream
Martin had a dream
Kendrick have a dream
[Chorus]
All my life I want money and power
Respect my mind or die from lead shower
I pray my dick get big as the Eiffel Tower
So I can fuck the world for seventy-two hours
[Verse 1]
Goddamn I feel amazin', damn, I'm in the Matrix
My mind is livin' on cloud nine and this 9 is never on vacation
Start up that Maserati and – vroom-vroom! – I'm racin'
Poppin' pills in the lobby and I pray they don't find her naked
And I pray you niggas is hatin',
shooters go after Judas
Jesus Christ, if I live life on my knees ain't no need to do this
Park it in front of Lueders, next to that Church's Chicken
All you pussies is losers, all my niggas is winners, screamin'—
[Chorus]
All my life I want money and power
Respect my mind or die from lead shower
I pray my dick get big as the Eiffel Tower
So I can fuck the world for seventy-two hours
[Post-Chorus 1]
Goddamn I got bitches, 

[Part 1: "Sing About Me"]
[Chorus]
When the lights shut off and it's my turn
To settle down, my main concern
Promise that you will sing about me
Promise that you will sing about me
I said when the lights shut off and it's my turn
To settle down, my main concern
Promise that you will sing about me
Promise that you will sing about me
[Verse 1]
I woke up this morning and figured I'd call you
In case I'm not here tomorrow
I'm hopin' that I can borrow
A peace of mind, I'm behind on what's really important
My mind is really distorted
I find nothing but trouble in my life
I'm fortunate you believe in a dream
This orphanage we call a ghetto is quite a routine
And last night was just another distraction
Or a reaction of what we consider madness
I know exactly what happened
You ran outside when you heard my brother cry for help
Held him like a newborn baby and made him feel
Like everything was alright in a fight he tried to put up
But the type of bullet that stuck
Had went against his will, that

[Chorus: Kendrick Lamar]
Miss me with that bullshit (Bullshit)
You ain't really wild, you a tourist (A tourist)
I be blackin' out with the purist (The purist)
I made a hundred thou', then I freaked it (I freaked it)
I made five-hundred-thou', then I freaked it (I freaked it)
I bought a '87 for the weekend (The weekend)
This ain't what you want, this ain't what you want (What you want)
This ain't what you want, this ain't what you want (What you want)
[Verse 1: Jay Rock]
And it's like that, lil' bitch
MVP, I don't get no sleep, no, I don't like that, lil' bitch
Bust that open, I want that ocean, yeah, that bite back, lil' bitch
Do it bite back, lil' bitch? Need two life jackets, lil' bitch
I ain't gon' hold you, I ain't gon' press you, never control you
I ain't gon' front you, keep it one-hundred, I don't know you
Boss like Top Dawg, bossed my life up, crossin' over
Stutter steppin', got a Hall of Fame in all my posters
I've been ready, my whip been ready, my bitch been ready, my clique

[Produced by Rahki]
​​
​
​​
​​
​​
​​
​​
​​
​​
​​
[Directed by Alexandre Moors and the Little Homies]
[Intro: Kendrick Lamar &
Ronald Isley
]
Is this mic on? (Hey, move this way, this way)
Hey, Hey! Hey! Turn the mic up, c'mon, c'mon
Is the mic on or not? I want the mic
We're bringing up nobody, nobody...
Nobody but the number one rapper in the world
He done traveled all over the world
He came back just to give you some game
All of the little boys and girls, come up here
(Ah, one two, one two, what's happening, fool?)
Come right here, this is for you, come on up
I done been through a whole lot (
Kendrick Lamar!
)
Trial, tribulation, but I know God
The Devil wanna put me in a bow tie (
Make some noise, brother!
)
Pray that the holy water don't go dry
As I look around me
So many motherfuckers wanna down me
But enemigo never drown me
In front of a dirty double-mirror they found me
[Chorus: Kendrick Lamar]
And (I love myself)
Huh, when you lookin' at me, tell me what do you see?
(I love mys

[Intro: Dale Warren]
Poverty's paradise
I don’t think I could find a way to make it on this earth
(I've been hungry all my life)
[Voicemail: Carl Duckworth]
What's up, family?
Yeah, it’s your cousin Carl, man, just givin' you a call, man
I know you been havin' a lot on yo' mind lately
And I know you feel like, you know
People ain't been prayin' for you
But you have to understand this, man, that we are a cursed people Deuteronomy 28:28 says, "The Lord shall smite thee with madness And blindness, and astonishment of heart"
See, family, that's why you feel like you feel
Like you got a chip on your shoulder
Until you follow His commandments, you gonna feel that way...
[Bridge 1: Charles Edward Sydney Isom Jr. &
Kendrick Lamar
]
Why God, why God do I gotta suffer?
Pain in my heart carry burdens full of struggle
Why God, why God do I gotta bleed?
Every stone thrown at you restin' at my feet
Why God, why God do I gotta suffer?
Earth is no more, won’t you burn this muh’fucka?
I don't think I c

...

[Verse 1: Kendrick Lamar]
I take a sip of Hennessy and then get pissy drunk
I ain't a drinker, I'm a thinker, call it what you want
But if you turn your back, know that you just missed your chance
To witness the realest shit that's ever been told to man
I found myself losing focus at a Sunday service
Embarrassed so I start questioning God, "What is my purpose?"
He said to live the way he did, that's all he want from me
Spread the word and witness, he rose on the first Sunday
I said alright, enthused that my Lord gave a listen
I opened my Bible in search to be a better Christian
And this from a person that never believed in religion
But shit, my life is so fucked up, man; I can't help but give in
I'm giving testimonies to strangers I never met
Hopped on the pulpit and told them how I was truly blessed
Felt like I'm free from all my sins when the service was over
Walked out the church, then got a call that my homie was murdered
Then lost my faith again
[Chorus: BJ The Chicago Kid]
What a

[Intro]
The hardest thing for me to do
Is to get you, to know me, within sixteen bars
That's the hardest thing
[Verse 1]
Who is K. Dot?
A young nigga from Compton
On the curb writing raps next to a gunshot
On the corners where the gangsters and the killers dwell
The fraudulent tender scars that get unveiled
Everyone I knew was either Crip or Piru
Cousins in elementary, relatives in high school
With that being said, each one of their rivals
Was aiming something at my head,
I needed survival
Got jumped, got jacked, shot at, shot back
And I don't even push a line, I'm just tryna push these rhymes
In the midst of staying neutral and discrete
My momma said you're judged by the company you keep
But what you can consider, that if it goes down
They'll kill you if you kill me
, it gets deep nigga
So if you ask what I'm doing
I'm tryna duck the influence of my city that's brewing, real talk, and
[Chorus]
This is why they fuck with me (real talk nigga, believe it)
This is why they fuck with me (d

[Intro]
Uh
A lil' Compton nigga
Just a lil' Compton nigga
One time for my niggas
You know Compton, nigga
Just a lil' Compton nigga
Kendrick Lamar
[Verse]
Back with a vengeance
Back with my heart and I'm venting
Back with another war pendant, salute me or shoot me
I never wore a kufi, ignore my religion
I believe there's one God, there's one love
And I believe if I fly, we all doves
I make a way for my people to see the light
Brighter than yellow bones
Me and my microphone could be one
Until I see BIG, Pac, see Pun but this ain't Eazy
I can't take a Big L, my city needs me, I gotta win
The proof is in the pudding
And I been fucking with Cosby since I was ten, you damn right
Any means necessary, get the campaign right
Very emotional I'm a Gemini, I love hard and I fight harder
, a born author
A born starter, in the game for real
And the games appeal to the struggling youth with no father
You can't learn if mistakes ain't made
You gotta pay your homage in order to get paid
You gotta pay a

[Produced by King Blue]
[Intro]
Oh what up, ho? Oh, what up?
I said oh, what up, ho? Oh, what up?
I said oh, what up, ho? Oh, what up?
Well, alright
[Chorus: Kendrick Lamar]
I'm going through something with life
But pussy and Patrón make you feel alright
Pussy and Patrón make you feel alright
Pussy and Patrón, that's some great advice
I'm going through something with life
But pussy and Patrón make you feel alright
Pussy and Patrón make you feel alright
Pussy and Patrón, that's some great advice
[Verse 1: Kendrick Lamar]
Welcome to my diary, stressing got me gray hairs
Something to inspire me, rather than society's
Woes, let me go, let me shine a lil' bit love
I want diamonds too, Ronnie on Player's Club
I used to have a 9 to 5, fresh out of school that was '05
That bitch was racist, got me fired, ever since then I had no job
Pushing in my mama van, stop for gas on Rosecrans
Trust me these niggas rushed me for something my cousin probably did
Guilty by association story of my life, nigg

[Verse]
It's been a week already, feeling weak already
Got me at peak, possibility of what could be
Situation is heavy, I've got to prove
On February 26th I lost my life too
It's like I'm here in a dark dream
Nightmare, hear screams recorded
Say that it sounds distorted but they know who it was
That was me yelling for help when he drowned in his blood
Why didn't he defend himself? Why couldn't he throw a punch?
And for our community do you know what this does?
Add to a trail of hatred
2012 was taped for the world to see
Set us back another 400 years
This is modern day slavery
The reason why I'm by your house
You threw your briefcase all on the couch
I plan on creeping through your damn door and blowing out
Every piece of your brain
'Til your son jumped in your arm
Cut off the engine then sped off in the rain
I'm on the dash with my bible in the trunk
Had the vodka on the top
How come I can long time
How come I could make it popular?
Puff on what I want
How come I could make it popular?

In [17]:
infile = open('namelist.pkl','rb')
artist_list = pickle.load(infile)
infile.close()

In [23]:
artist_list_np = np.array(artist_list)
artist_list_np.shape

(2992,)

In [26]:
for name in artist_list:
    write_lyrics_to_file(str(name), 100)

Found 100 songs by Adele
[Verse 1]
Hello, it's me
I was wondering if after all these years you'd like to meet
To go over everything
They say that time's supposed to heal ya, but I ain't done much healing
Hello, can you hear me?
I'm in California dreaming about who we used to be
When we were younger and free
I've forgotten how it felt before the world fell at our feet
[Pre-Chorus]
There's such a difference between us
And a million miles
[Chorus]
Hello from the other side
I must've called a thousand times
To tell you I'm sorry for everything that I've done
But when I call, you never seem to be home
Hello from the outside
At least, I can say that I've tried
To tell you I'm sorry for breaking your heart
But it don't matter, it clearly doesn't tear you apart anymore
[Verse 2]
Hello, how are you?
It's so typical of me to talk about myself, I'm sorry
I hope that you're well
Did you ever make it out of that town where nothing ever happened?
[Pre-Chorus]
It's no secret that the both of us
Are r

[Produced by Fraser T Smith]
[Verse 1]
I let it fall, my heart
And as it fell, you rose to claim it
It was dark, and I was over
Until you kissed my lips and you saved me
My hands, they were strong
But my knees were far too weak
To stand in your arms
Without falling to your feet
[Pre-Chorus]
But there's a side to you
That I never knew, never knew
All the things you'd say
They were never true, never true
And the games you'd play
You would always win, always win
[Chorus]
But I set fire to the rain
Watched it pour as I touched your face
Well, it burned while I cried
'Cause I heard it screamin' out your name
Your name
[Verse 2]
When I lay with you
I could stay there, close my eyes
Feel you here forever
You and me together, nothing is better
[Pre-Chorus]
'Cause there's a side to you
That I never knew, never knew
All the things you'd say
They were never true, never true
And the games you'd play
You would always win, always win
[Chorus]
But I set fire to the rain
Watched it pour as I touched y

[Verse 1]
This is the end
Hold your breath and count to ten
Feel the Earth move, and then
Hear my heart burst again
For this is the end
I've drowned and dreamt this moment
So overdue, I owe them
Swept away, I'm stolen
[Chorus]
Let the sky fall
When it crumbles
We will stand tall
Face it all together
Let the sky fall
When it crumbles
We will stand tall
Face it all together
At Skyfall
At Skyfall
[Verse 2]
Skyfall is where we start
A thousand miles and poles apart
Where worlds collide and days are dark
You may have my number
You can take my name
But you'll never have my heart
[Chorus]
Let the sky fall (Let the Skyfall)
When it crumbles (When it crumbles)
We will stand tall (We will stand tall)
Face it all together
Let the sky fall (Let the Skyfall)
When it crumbles (When it crumbles)
We will stand tall (We will stand tall)
Face it all together
At Skyfall
Let the sky fall
When it crumbles
We will stand tall
Let the sky fall
When it crumbles
We will stand tall
[Bridge]
Where you go I go
Wha

[Verse 1]
When will I see you again?
You left with no goodbye, not a single word was said
No final kiss to seal any sins
I had no idea of the state we were in
[Pre-Chorus]
I know I have a fickle heart
and a bitterness
And a wandering eye,
and a heaviness in my head
[Chorus]
But don't you remember?
Don't you remember?
The reason you loved me before
Baby, please remember me once more
[Verse 2]
When was the last time you thought of me?
Or have you completely erased me from your memory?
I often think about where I went wrong
The more I do, the less I know
[Pre-Chorus]
But I know I have a fickle heart
and a bitterness
And a wandering eye,
and a heaviness in my head
[Chorus]
But don't you remember?
Don't you remember?
The reason you loved me before
Baby, please remember me once more
[Bridge]
Gave you the space so you could breathe
I kept my distance so you would be free
And hope that you find the missing piece
To bring you back to me
[Chorus]
Why don't you remember?
Don't you remember?
The r

KeyboardInterrupt: 